In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

def extract_text_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        for script_or_style in soup(['script', 'style']):
            script_or_style.decompose()
        text = soup.get_text(separator=' ')
        #Limpar texto
        lines = (line.strip() for line in text.splitlines())
        parts = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(part for part in parts if part)
        return text

In [ ]:
text_extracted = extract_text_from_url('https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo')
#

In [ ]:
subscription_key = "YOUR_SUBSCRIPTION_KEY"
endpoint = "YOUR_ENDPOINT"
location = "YOUR_LOCATION"
target_language = "pt-br"

In [ ]:
def translator_text(text_extracted, target_language):
  path = '/translate'
  constructed_url = location + path
  headers = {
      'Ocp-Apim-Subscription-Key': subscription_key,
      'Ocp-Apim-Subscription-Region': endpoint,
      'Content-type': 'application/json',
      'X-ClientTraceId': str(os.urandom(16))
  }

  body = [{
      'text': text_extracted
  }]

  params = {
      'api-version': '3.0',
      'from': 'en',
      'to': target_language
  }
  request = requests.post(constructed_url, params=params, json=body, headers=headers)
  response = request.json()
  return response[0]["translations"][0]["text"]

In [ ]:
# Função para traduzir o artigo a partir de uma URL
def translate_article(url):

    # Extraindo o texto do artigo
    try:
        texto_original = extract_text_from_url(url)

        # Fazendo a tradução
        texto_traduzido = translator_text(text_extracted, target_language)

        # Retornar em Markdown
        texto_markdown = f"""# Artigo Traduzido\n\n{texto_traduzido}"""
        print(texto_markdown)
    except requests.exceptions.RequestException as e:
        print(f"Erro ao processar a requisição: {e}")

In [ ]:
url = 'https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo'
article = translate_article(url)
print(article)